In [9]:
import pandas as pd
from glob import glob
import plotly.express as px
import numpy as np
import os

In [23]:
csv_list = glob('date_category_stm\*.csv')    
cate_list = []
for i in csv_list:
    cate_name = i.split('_')[-1].split('.')[0]
    cate_list.append(cate_name)
    os.makedirs('국민청원/templates/'+cate_name, exist_ok=True)
    csv = pd.read_csv(i, encoding='cp949')
    monthpertopic = {}
    for i in range(0, len(csv)):
        date = csv.loc[i]['covariate.value']
        topic = csv.loc[i]['topic']
        estimate = csv.loc[i]['estimate']
        if date in monthpertopic:
            if topic in monthpertopic[date]:
                a = 1
            else:
                monthpertopic[date][topic] = estimate
        else:
            tmp = {}
            tmp[topic] = estimate
            monthpertopic[date] = tmp
    
    a = []
    for key, val in monthpertopic.items():
        for key2, val2 in val.items():
            tmp = [key, key2, val2]
            key = str(key)
            v = (int(key[:4]) - 2020) * 12 + (int(key[4:6]))
            tmp.append(v)
            dd = key[:4]+ '-' + key[4:6] + '-' + '01'
            tmp.append(dd)
            a.append(tmp)
            
    c = ['month', 'topicnum', 'est', 'index', 'date']
    df_list = pd.DataFrame(a, columns=c)
    
    df_list = df_list.sort_values(by=['date'], axis=0)
    
    
    df_list['est'] = np.exp(np.log(df_list['est']) + 10)
    
    
    for i in range(1, 26):
        a = []
        for key, val in monthpertopic.items():
            for key2, val2 in val.items():
                if key2 == i:
                    tmp = [key, key2, val2]
                    key = str(key)
                    # v = (int(key[:4]) - 2020) * 12 + (int(key[4:6]))
                    v = key[:4]+ '-' + key[4:6] + '-' + '01'
                    tmp.append(v)
                    a.append(tmp)
        c = ['month', 'topicnum', 'est', 'index']
        df_list = pd.DataFrame(a, columns=c)
        # df_list['est'] = np.exp(np.log(df_list['est']) + 10)
        df_list = df_list.sort_values(by=['index'], axis=0)
        fig = px.line(df_list, x="index", y="est", title='month topic', range_x=['2019-12-01','2022-01-01'])
        fig.update_layout(
            title=' ')
        fig.update_xaxes(showgrid=False)
        fig.update_yaxes(showgrid=False)
        fig.write_html("국민청원/templates/"+cate_name+"/" + str(i) +"file.html")


In [25]:
for i in cate_list:
    print(i)
print(cate_list, end='\n')

경제민주화
교통건축국토
기타
농산어촌
문화예술체육언론
미래
반려동물
보건복지
성장동력
안전환경
외교통일국방
육아교육
인권성평등
일자리
저출산고령화대책
정치개혁
행정
['경제민주화', '교통건축국토', '기타', '농산어촌', '문화예술체육언론', '미래', '반려동물', '보건복지', '성장동력', '안전환경', '외교통일국방', '육아교육', '인권성평등', '일자리', '저출산고령화대책', '정치개혁', '행정']


In [45]:
import plotly.express as px

fig = px.scatter(df_list, x="date", y="topicnum",
	         size="est", color="topicnum",
                 hover_name="topicnum", log_x=False, size_max=13
                )
fig.update_layout(
    title='행정 토픽 빈도')
fig.show()
fig.write_html("./file.html")